In [ ]:
import pandas as pd
import timeit

#Carregando os dados
df = pd.read_csv('C:\projetos-ds\python-data-science-fundamentals\data\df_produtos_500k.csv')
print(f'Dados carregados: {len(df)} linhas.')

# --- MÉTODO 1: LOOP FOR TRADICIONAL ---
def calcular_faturamento_loop(df):
    faturamento = []
    # Utiliza o zip para iterar sobre as colunas com performance aceitável
    for preco, qtd in zip(df['preco_unitario'], df['quantidade_vendida']):
        faturamento.append(preco * qtd)
    return faturamento

# Medição do tempo
tempo_loop = timeit.timeit(
    lambda: calcular_faturamento_loop(df),
    number=10 # Repetimos 10 vezes para uma média mais estável
)
tempo_medio_loop = tempo_loop / 10
print(f"\nTempo médio (Loop FOR): {tempo_medio_loop:.5f} segundos")

# --- MÉTODO 2: LIST COMPREHENSION (SUA TAREFA) ---
def calcular_faturamento_comprehension(df):
    # A List Comprehension deve substituir o loop for acima em uma única linha
    faturamento = [preco * qtd for preco, qtd in zip(df['preco_unitario'], df['quantidade_vendida'])]
    return faturamento

# Medição do tempo
tempo_comprehension = timeit.timeit(
    lambda: calcular_faturamento_comprehension(df),
    number=10
)
tempo_medio_comprehension = tempo_comprehension / 10
print(f"Tempo médio (List Comprehension): {tempo_medio_comprehension:.5f} segundos")